In [1]:
!pip install huggingface_hub boto3

  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/93/27/1fb384a841e9661faad1c31cbfa62864f59632e876df5d795234da51c395/huggingface_hub-0.30.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 11.0 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import boto3
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download

In [6]:
!pwd
!mkdir -p /opt/app-root/src/llm

/opt/app-root/src


In [27]:
bucket_name = "llm"


def upload_file_to_minio(repo_id, prefix_key, fullfilename=None):
    path = f"/opt/app-root/src/llm/{prefix_key}" if fullfilename is None else fullfilename
    snapshot_download(repo_id=repo_id, cache_dir=path, token='your huggingface token')
    for filename in os.listdir(path):
        fullfilename = path + '/' + filename
        print("Uploading:", fullfilename)
        if os.path.isdir(fullfilename):
            upload_file_to_minio(repo_id, prefix_key, fullfilename)
        else:
            try:
                s3_client.upload_file(fullfilename, bucket_name, prefix_key + '/' + filename)
            except Exception as e:
                print(f"An error occurred while uploading {fullfilename}: {e}")


upload_file_to_minio('meta-llama/Llama-3.1-8B-Instruct', "llama")
upload_file_to_minio('ibm-granite/granite-3.0-8b-instruct', "granite")

Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/refs
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/refs/main
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/snapshots
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/snapshots/0e9e39f249a16976918f6564b8830bc894c89659
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/snapshots/0e9e39f249a16976918f6564b8830bc894c89659/.gitattributes
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/snapshots/0e9e39f249a16976918f6564b8830bc894c89659/model-00002-of-00004.safetensors
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Llama-3.1-8B-Instruct/snapshots/0e9e39f249a16976918f6564b8830bc894c89659/USE_POLICY.md
Uploading: /opt/app-root/src/llm/llama/models--meta-llama--Ll

In [17]:
!rm -rf /opt/app-root/src/llm/